In [4]:
# import
import gym
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Add, Multiply
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

import tensorflow as tf
import random
from collections import deque

In [7]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
class ActorCritic:
    def __init__(self, env, sess):
        self.env = env
        self.sess = sess
        
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_decay = .995
        self.gamma = .95
        self.tau = .125
        
        self.memory = deque(maxlen=2000)
        self.actor_state_input, self.actor_model = self.create_actor_model()
        _, self.target_actor_model = self.create_actor_model()
        
        self.actor_critic_grad = tf.placeholder(tf.float32, [None, self.env.action_space,shape[0]])
        
        actor_model_weights = self.actor_model.trainable_weights
        self.actor_grads = tf.gradients(self.actor_model.output, 
                                       actor_model_weights, -self.actor_critic_grad)
        grads = zip(self.actor_grads, actor_model_weights)
        self.optimize = tf.train.AdamOptimizer(self.learning_rate).apply_gradients(grads)
        
        self.acritic_state_input, self.critic_action_input, self.critic_model = self.create_critic_model()
        _, _, self.target_critic_model = self.create_critic_model()
        
        self.critic_grads = tf.gradients(self.critic_model.output, self.critic_action_input)
        
        self.sess.run(tf.initialize_all_variables())
        
        def create_actor_model(self):
            state_input = Input(shape = self.env.observation_space.shape)
            h1 = Dense(24,activation='relu')(state_input)
            h2 = Dense(48, activation = 'relu')(h1)
            h3 = Dense(24, activation = 'relu')(h2)
            output = Dense(self.env.action_space.shape[0], activation='relu')(h3)
            
            model = Model(input=state_input, output = output)
            adam = Adam(lr=0.001)
            model.compile(loss="mse", optimizer=adam)
            return state_input, model
        
        def create_critic_model(self):
            state_input = Input(shape=self.env.observaion_space.shape)
            state_h1 = Dense(24, activation='relu')(state_input)
            state_h2 = Dense(48)(state_h1)
            
            action_input = Input(shape=self.env.action_space.shape)
            action_h1 = Dense(48)(action_input)
            
            merged = Add()([state_h2, action_h1])
            merged_h1 = Dense(24, activation = 'relu')(merged)
            output = Dense(1, activation='relu')(merged_h1)
            model = Model(input=[state_input, action_input], output=output)
            
            adam = Adam(lr=0.001)
            model.compile(loss="mse", optimizer=adam)
            return state_input, action_input, model
        
        def remember(self,cur_state, action, reward, new_state,done):
            self.memory.append([cur_state, action,reward, new_state,done])
            
        def _train_actor(self, samples):
            for sample in samples:
                cur_state, action,reward,new_state,_ = sample
                predicted_action = self.actor_model.predict(cur_state)
                grads = self.sess.run(self.critic_grads, feed_dict={
                    self.critic_state_input : cur_state,
                    self.critic_action_input : predicted_action
                })[0]
                
                self.sess.run(self.optimize, feed_dict={
                    self.actor_state_input : cur_state,
                    self.actor_critic_grad: grads
                })
                
        def _train_critic(self, samples):
            for sample in samples:
                cur_state,action, reward, new_state, done = smaple
                if not done:
                    target_action = self.targer_actor_model.predict(new_state)
                    future_reward = self.target_critic_model.predict(
                        [new_state, target_action])[0][0]
                    reward += self.gamma * future_reward
                self.critic_model.fit([cur_state, action], reward, verbose=0)
         
        def train(self):
            batch_size = 32
            if len(self.memory) < batch_size:
                return
            rewards = []
            samples = random.sample(self.memorym, batch_size)
            self._train_critic(samples)
            self._train_actor(samples)
            
        
        def _update_actor_target(self):
            actor_model_weights = self.actor_model.get_weights()
            actor_target_wegihts = self.target_critic_model.get_weights()
            
            for i in range(len(actor_target_weights)):
                actor_target_weights[i] = actor_model_weights[i]
            self.target_critic_model.set_weights(actor_target_weights)
            
        def _update_critic_target(self):
            critic_model_weights = self.critic.model.get_weights()
            critic_target_weights = self.critic_target_model.get_weights()
            
            for i in range(len(critic_target_weights)):
                critic_target_weights[i] = ciritic_model_weights[i]
            self.critic_target_model.set_weights(critic_target_weights)
            
        def update_target(self):
            self._update_actor_target()
            self._update_critic_target()
            
        def act(self,cur_state):
            self.epsilon *= self.epsilon_decay
            if np.random.random() < self.epsilon:
                return self.env.action_space.smaple()
            return self.actor_model.predict(cur_state)
        
def main():
    sess = tf.Session()
    K.set_session(sess)
    env = gym.make("Pendulum-v0")
    actor_critic = ActorCritic(env,sess)
    
    num_trials = 10000
    trial_len = 500
    
    cur_state = env.reset()
    action = env.action_space.sample()
    
    while True:
        env.render()
        cur_state = cur_state.reshape((1,env.observation_space.shape[0]))
        action = actor_critic.act(cur_state)
        action = action.reshape((1,env,action_space.shape[0]))
        
        new_state, reward,done, _ = env.step(action)
        new_state = new_state.reshape((1, env.observation_space.shape[0]))
        
        actor_critic.remember(cur_state, action, reward, new_state, done)
        actor_critic.train()
        
        cur_state = new_state
        
if __name__ == "__main__":
    main()

2022-12-09 11:41:50.104681: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-09 11:41:50.134814: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3593190000 Hz
2022-12-09 11:41:50.135511: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x3f54050 executing computations on platform Host. Devices:
2022-12-09 11:41:50.135529: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
2022-12-09 11:41:50.135747: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
ERROR:root:Internal Python error in the inspect module.
Below is

Traceback (most recent call last):
  File "/home/soohyoen/anaconda3/envs/rein_study/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_11383/1516610257.py", line 150, in <module>
    main()
  File "/tmp/ipykernel_11383/1516610257.py", line 125, in main
    K.set_session(sess)
AttributeError: module 'tensorflow_core.keras.backend' has no attribute 'set_session'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/soohyoen/anaconda3/envs/rein_study/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'AttributeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/soohyoen/anaconda3/envs/rein_study/lib/python3